# This script annotates transcripts of video simulation role-playing exercises from job candidates using large language models

In [ ]:
# Install libraries that might not be installed as necessary (adjust manually)
# !pip install openai
# !pip install pandas

In [6]:
# Import necessary libraries
import openai          # OpenAI API for AI model access
from openai import OpenAI  # The main client class for interacting with OpenAI's API
import requests        # HTTP library for making web requests (API calls, downloading data, etc.)
import re              # Regular expressions
import os              # Operating system interface
import pandas as pd    # DataFrames for tabular data manipulation and analysis
import numpy as np     # Numerical computing
import time            # Time-related functions
import random          # Random number generation

# set seed for reproducibility
random.seed(1337)  # Sets seed for Python's random module

In [ ]:
# Load data
# Advisement 2
advise = pd.read_csv("data_advise_text.csv")
# advise = advise.iloc[:2] # keep only first two rows for pilot data analysis purposes
# advise.head()

# # Change management 2
# change_manage = pd.read_csv("data_change_manage_text.csv")
# # Team management 2
# team_manage = pd.read_csv("data_team_manage_text.csv")
# # Team work 2
# team_work = pd.read_csv("data_team_work_text.csv")

In [ ]:
# Import OpenAI API key
client = OpenAI(api_key="enter_key_here") # key hidden for privacy

## Function to get ratings from openai API

In [17]:
# Manually change `model` to another openai model (as desired)
def get_rating(client, text, prompt_template, model="gpt-4o"):
    """
    Query the OpenAI model for a 1–5 rating based on a formatted prompt with transcript text.
    
    Parameters:
    - client: OpenAI client instance
    - text (str): The transcript to evaluate
    - prompt_template (str): A prompt string with {text} placeholder for transcript
    - model (str): Model name (default: gpt-4o)

    Returns:
    - int or None: Extracted rating or None if parsing fails
    """
    try:
        full_prompt = prompt_template.format(text=text.strip())

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": full_prompt},
                {"role": "user", "content": ""}  # no additional input
            ],
            temperature=0.7, # lower values reduce randomness (0.7 matches the ChatGPT behavior)
            max_tokens=5  # we expect only a single digit
        )

        rating_text = response.choices[0].message.content.strip()
        match = re.search(r'\b([1-5](?:\.\d)?)\b', rating_text)
        if match:
            return int(match.group(0))
        else:
            print(f"Warning: Non-numeric or out-of-range response: '{rating_text}'")
            return None

    except Exception as e:
        print(f"Error processing text: {e}")
        return None

# Prompts

In [18]:
# Define general instructions for competence annotation
context = """
**Context of the role-playing exercise:**
You are a professional recruiter. Your task is to evaluate a job candidate based on their responses 
in a role-playing exercise. In this exercise, the candidate responds verbally to a fictional colleague named Lara, across four scenes. 
Lara shares professional frustrations, mistakes with client projects, emotional exhaustion, 
and personal financial concerns, and she asks the candidate for support and advice. In summary:

- **Scene 1**: Lara expresses frustration about being overlooked at work, doing repetitive tasks, 
and lacking opportunities to grow. She asks for advice.
- **Scene 2**: Lara confesses she mishandled two client projects, causing cost issues and customer dissatisfaction. 
She is afraid to tell the team leader and asks if the candidate can speak on her behalf.
- **Scene 3**: Lara reveals emotional exhaustion and financial stress due to her daughter’s chronic illness. 
She wonders if she can get extra leave or if the candidate can take over some tasks.
- **Scene 4**: Lara feels slightly better but still overwhelmed. She asks the candidate for concrete advice on what to do next.
"""

instructions = """
**Candidate Response:**
You are given the transcript of the candidate’s full response across all four scenes below.

**Evaluation Instructions:**
1. Read the transcript of the job candidate's reponse (delimeated between the triple backticks).
2. Focus on **verbal behaviors, values, or attitudes** that reflect this competency.
3. Rate the candidate on a **1 to 5 scale**, using the following rubric:

- **1** = Poor  
- **2** = Fair  
- **3** = Satisfactory  
- **4** = Good  
- **5** = Excellent

**Output only a single number from 1 to 5. Do not provide any explanation.**

```{text}```
"""


# Prompts for rating job competencies
promptAD_PROBL_I1 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate distinguishes between main and side issues** in Lara’s story.
{instructions}
"""

promptAD_PROBL_I2 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate makes connections between different aspects of the issue** in Lara’s story.
{instructions}
"""

promptAD_PROBL_I3 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate identifies the core of the information** in Lara’s story.
{instructions}
"""

promptAD_CREAT_I1 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate comes up with original ideas** as a response to Lara's story.
{instructions}
"""

promptAD_CREAT_I2 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate approaches matters from multiple perspectives when generating solutions/ideas** as a response to Lara's story.
{instructions}
"""

promptAD_CREAT_I3 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate combines existing elements into something new** in their response to Lara's story.
{instructions}
"""

promptAD_OORDE_I1 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate distinguishes between relevant and irrelevant aspects** in their response to Lara's story.
{instructions}
"""

promptAD_OORDE_I2 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate supports their reasoning with facts** in their response to Lara's story.
{instructions}
"""

promptAD_OORDE_I3 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate states both pros and cons of their own standpoint** in their response to Lara's story.
{instructions}
"""

promptAD_ORGANS_I1 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate takes into account the sensitivities of others in the organization during the conversation** in their response to Lara's story.
{instructions}
"""

promptAD_ORGANS_I2 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate checks in advance how actions/decisions will be received within the organization** in their response to Lara's story.
{instructions}
"""

promptAD_ORGANS_I3 = f"""
{context}
**Competency to evaluate:**  
Evaluate **to what extent the candidate investigates the consequences for other people/organizational units before making a decision** in their response to Lara's story.
{instructions}
"""

In [19]:
# Initialize new columns to store the annotations

# Specify which LLM model is being used for annotations
# This will be prefixed to all generated column names (e.g., "4o_rating607A")
MODEL_USED = "4o"  # Options: "4o", "R1", "gpt4", etc.

# Dictionary mapping source columns to their respective rating columns and prompts
annotation_config = {
    "Beoordeling.all": [
        (f"{MODEL_USED}_AD_PROBL_I1", promptAD_PROBL_I1),
        (f"{MODEL_USED}_AD_PROBL_I2", promptAD_PROBL_I2),
        (f"{MODEL_USED}_AD_PROBL_I3", promptAD_PROBL_I3),
        (f"{MODEL_USED}_AD_CREAT_I1", promptAD_CREAT_I1),
        (f"{MODEL_USED}_AD_CREAT_I2", promptAD_CREAT_I2),
        (f"{MODEL_USED}_AD_CREAT_I3", promptAD_CREAT_I3),
        (f"{MODEL_USED}_AD_OORDE_I1", promptAD_OORDE_I1),
        (f"{MODEL_USED}_AD_OORDE_I2", promptAD_OORDE_I2),
        (f"{MODEL_USED}_AD_OORDE_I3", promptAD_OORDE_I3),
        (f"{MODEL_USED}_AD_ORGANS_I1", promptAD_ORGANS_I1),
        (f"{MODEL_USED}_AD_ORGANS_I2", promptAD_ORGANS_I2),
        (f"{MODEL_USED}_AD_ORGANS_I3", promptAD_ORGANS_I3)
    ]
}

# Initialize all rating columns with NaN values
# Creates columns in format: "{MODEL_USED}_ratingXXX"
for ratings in annotation_config.values():
    for rating_col, _ in ratings:
        advise[rating_col] = np.nan

# Verify new columns were created successfully
# display(advise.head())


# Text annotation

In [20]:
# Process each row in the advise DataFrame
for idx, row in advise.iterrows():
    try:
        # Loop through each Beoordeling column and its associated ratings/prompts
        for beoordeling_col, ratings in annotation_config.items():
            
            # Extract the transcript
            text = row.get(beoordeling_col, "")
            
            if pd.notna(text) and text.strip() != "":
                for rating_col, prompt_template in ratings:
                    
                    # Query the model with formatted prompt
                    rating = get_rating(client, text, prompt_template)
                    
                    # Save the rating
                    advise.loc[idx, rating_col] = rating
                    
                    # Add small delay to respect rate limits
                    time.sleep(0.5)

    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        continue

    # Progress logging
    if (idx + 1) % 5 == 0:
        print(f"Processed {idx + 1}/{len(advise)} rows")


In [ ]:
# Calculate the average for each of the four sub-competencies and store in new columns
advise["4o_AD_PROBL"] = advise[["4o_AD_PROBL_I1", "4o_AD_PROBL_I2", "4o_AD_PROBL_I3"]].mean(axis=1)
advise["4o_AD_CREAT"] = advise[["4o_AD_CREAT_I1", "4o_AD_CREAT_I2", "4o_AD_CREAT_I3"]].mean(axis=1)
advise["4o_AD_OORDE"] = advise[["4o_AD_OORDE_I1", "4o_AD_OORDE_I2", "4o_AD_OORDE_I3"]].mean(axis=1)
advise["4o_AD_ORGANS"] = advise[["4o_AD_ORGANS_I1", "4o_AD_ORGANS_I2", "4o_AD_ORGANS_I3"]].mean(axis=1)

# Calculate the average of the 4 new columns
advise["4o_AD"] = advise[["4o_AD_PROBL", "4o_AD_CREAT", "4o_AD_OORDE", "4o_AD_ORGANS"]].mean(axis=1)

# advise.head()

In [ ]:
# Optional: Save results
# advise.to_csv('annotated_advise.csv', index=False)

## Codebook

The codebook below explains the column names of the datasets.  
The original column names (e.g., `Score.566`) have been re-named such that the new names indicate:  

**(a)** the job competency being evaluated (i.e., `AD` = Advisement, `CM` = Change management, `TM` = Team management, `TW` = Team work),  

**(b)** the sub-dimension being evaluated (e.g., `PROBM` = Problem analysis),  

**(c)** the number of the item of the subdimension (e.g., `I1` = item 1).  

For instance, in the column name `AD_PROBL_I1`, `AD` refers to the competency of `Advisement`, `PROBL` refers to the sub-competence of `Problem analyse`, and `I1` refers to the first item (item 1) of that sub-compentence.


**Advisement Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
|Beoordeling.607 |Beoordeling.607 | Transcript scene2 |
|Beoordeling.608 |Beoordeling.608 | Transcript scene3 |
|Beoordeling.609 |Beoordeling.609 | Transcript scene4 |
|Beoordeling.610 |Beoordeling.610 | Transcript scene5 |
|Beoordeling.all |Beoordeling.all | Transcripts scene2 to scene5 |
| AD_PROBL_I1 | Score.566   | Advisor's assessment: PROBL-I1_Maakt onderscheid tussen hoofd- en bijzaken in de situatie |
| AD_PROBL_I2 | Score.567   | Advisor's assessment: PROBL-I2_Legt verbanden tussen verschillende aspecten van de vraagstelling |
| AD_PROBL_I3 | Score.568   | Advisor's assessment: PROBL-I3_Benoemt de kern van de informatie |
| AD_CREAT_I1 | Score.569   | Advisor's assessment: CREAT-I1_Komt met originele ideeën |
| AD_CREAT_I2 | Score.570   | Advisor's assessment: CREAT-I2_Benadert zaken vanuit meerdere invalshoeken bij het bedenken van oplossingen/ideeën |
| AD_CREAT_I3 | Score.571   | Advisor's assessment: CREAT-I3_Combineert bestaande zaken tot iets nieuws |
| AD_OORDE_I1 | Score.1544 | Advisor's assessment: OORDE-I1_Maakt in zijn/haar/hen beeldvorming een onderscheid tussen relevante en niet relevante aspecten van een kwestie |
| AD_OORDE_I2 | Score.1545 | Advisor's assessment: OORDE-I2_Onderbouwt zijn/haar/hen redenering met feiten |
| AD_OORDE_I3 | Score.1546 | Advisor's assessment: OORDE-I3_Benoemt van eigen standpunt zowel voor- als nadelen |
| AD_ORGANS_I1 | Score.1547 | Advisor's assessment: ORGANS-I1_Houdt in het gesprek rekening met gevoeligheden van anderen in de organisatie |
| AD_ORGANS_I2 | Score.1548 | Advisor's assessment: ORGANS-I2_Toetst vooraf hoe acties/besluiten binnen de organisatie ontvangen zullen worden |
| AD_ORGANS_I3 | Score.1549 | Advisor's assessment: ORGANS-I3_Onderzoekt de gevolgen voor andere personen/organisatie-onderdelen alvorens te besluiten |


**Change management Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
| CM_LEIDIN_I2 | Score.615  | Oordeel adviseur: LEIDIN-I2_Geeft aandacht aan goede prestaties van de medewerker, uit waardering |
| CM_LEIDIN_I1 | Score.616  | Oordeel adviseur: LEIDIN-I1_Schept helderheid over de van de medewerker verwachte prestaties |
| CM_LEIDIN_I3 | Score.617  | Oordeel adviseur: LEIDIN-I3_Past stijl van leidinggeven aan de situatie en persoon aan |
| CM_COACH_I1 | Score.618  | Oordeel adviseur: COACH-I1_Helpt de ander inzicht te krijgen in diens sterke en zwakke punten |
| CM_COACH_I2 | Score.619  | Oordeel adviseur: COACH-I2_Stimuleert de ander om zich te ontwikkelen en nieuwe uitdagingen aan te gaan |
| CM_COACH_I3 | Score.620  | Oordeel adviseur: COACH-I3_Geeft ruimte om op een eigen manier te werken aan ontwikkeling |
| CM_VISIEU_I1 | Score.627  | Oordeel adviseur: VISIEU-I1_Benoemt de (lange termijn) doelen voor de organisatie |
| CM_VISIEU_I2 | Score.628  | Oordeel adviseur: VISIEU-I2_Geeft inzicht heeft in wat het beleid betekent voor de werkzaamheden |
| CM_VISIEU_I3 | Score.629  | Oordeel adviseur: VISIEU-I3_Legt meer de nadruk op mogelijkheden in de toekomst dan op beperkingen 
| CM_RESUL_I1 | Score.621  | Oordeel adviseur: RESUL-I1_Komt met voorstellen over hoe doelen sneller of beter kunnen worden bereikt |
| CM_RESUL_I2 | Score.622  | Oordeel adviseur: RESUL-I2_Maakt duidelijke afspraken met de medewerker over de te bereiken resultaten |
| CM_RESUL_I3 | Score.623  | Oordeel adviseur: RESUL-I3_Grijpt tijdig in wanneer resultaten niet bereikt dreigen te worden; stuurt bij |

**Team management Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
| TM_LEIDIN_I2 | Score.615  | Oordeel adviseur: LEIDIN-I2_Geeft aandacht aan goede prestaties van de medewerker, uit waardering |
| TM_LEIDIN_I1 | Score.616  | Oordeel adviseur: LEIDIN-I1_Schept helderheid over de van de medewerker verwachte prestaties |
| TM_LEIDIN_I3 | Score.617  | Oordeel adviseur: LEIDIN-I3_Past stijl van leidinggeven aan de situatie en persoon aan |
| TM_COACH_I1 | Score.618  | Oordeel adviseur: COACH-I1_Helpt de ander inzicht te krijgen in diens sterke en zwakke punten |
| TM_COACH_I2 | Score.619  | Oordeel adviseur: COACH-I2_Stimuleert de ander om zich te ontwikkelen en nieuwe uitdagingen aan te gaan |
| TM_COACH_I3 | Score.620  | Oordeel adviseur: COACH-I3_Geeft ruimte om op een eigen manier te werken aan ontwikkeling |
| TM_OVERT_I1 | Score.552  | Oordeel adviseur: OVERT-I1_Gebruikt argumenten die aansluiten bij de belangen of voorkeuren van de medewerker |
| TM_OVERT_I2 | Score.553  | Oordeel adviseur: OVERT-I2_Maakt duidelijk wat de voordelen van zijn/haar/diens voorstel zijn voor de medewerker |
| TM_OVERT_I3 | Score.554  | Oordeel adviseur: OVERT-I3_Maakt weerstand van de medewerker bespreekbaar |
| TM_RESUL_I1 | Score.621  | Oordeel adviseur: RESUL-I1_Komt met voorstellen over hoe doelen sneller of beter kunnen worden bereikt |
| TM_RESUL_I2 | Score.622  | Oordeel adviseur: RESUL-I2_Maakt duidelijke afspraken met de medewerker over de te bereiken resultaten |
| TM_RESUL_I3 | Score.623  | Oordeel adviseur: RESUL-I3_Grijpt tijdig in wanneer resultaten niet bereikt dreigen te worden; stuurt bij |


**Team work Codebook**:
| Code_name | Original_column_name | Label |
| -------- |-------------|-------|
| TW_SAMEN_I1 | Score.549  | Oordeel adviseur: SAMEN-I1_Biedt de collega hulp aan |
| TW_SAMEN_I2 | Score.550  | Oordeel adviseur: SAMEN-I2_Stelt gezamenlijk belang boven het eigen belang |
| TW_SAMEN_I3 | Score.551  | Oordeel adviseur: SAMEN-I3_Onderneemt acties die de samenwerking bevorderen |
| TW_OVERT_I1 | Score.552  | Oordeel adviseur: OVERT-I1_Maakt duidelijk wat de voordelen zijn van zijn/haar/diens voorstel |
| TW_OVERT_I2 | Score.553  | Oordeel adviseur: OVERT-I2_Gebruikt argumenten die aansluiten bij de belangen van de ander |
| TW_OVERT_I3 | Score.554  | Oordeel adviseur: OVERT-I3_Maakt weerstand van de ander op gepaste manier bespreekbaar |
| TW_MONDEC_I1 | Score.555  | Oordeel adviseur: MONDEC-I1_Brengt de boodschap op een voor de ander begrijpelijke manier over |
| TW_MONDEC_I2 | Score.556  | Oordeel adviseur: MONDEC-I2_Verduidelijkt de boodschap met concrete voorbeelden |
| TW_MONDEC_I3 | Score.557  | Oordeel adviseur: MONDEC-I3_Onderscheidt in zijn/haar/diens boodschap hoofd- en bijzaken op logische manier |
| TW_ORGANE_I1 | Score.558  | Oordeel adviseur: ORGANE-I1_Stelt prioriteiten voor het eigen werk; belangrijke zaken eerst |
| TW_ORGANE_I2 | Score.559  | Oordeel adviseur: ORGANE-I2_Houdt zich in het eigen werk aan met anderen gemaakte afspraken |
| TW_ORGANE_I3 | Score.560  | Oordeel adviseur: ORGANE-I3_Laat tijdig aan de collega weten wanneer het eigen werk geen nieuwe bezigheden toelaat |
